# Setting up a single simulation pair


### Initial simulation grid

We have 6 simulation parameters: 

$$ H = \{ \mu_H, \mu_{He}, \Omega_\star, \Delta_p^2, n_p, \alpha_p \} $$

We will choose the a parameter volume for the simulation similar to the one below:
 - $ 0.5 < \mu_H < 2.0 $
 - $ 0.5 < \mu_{He} < 2.0 $
 - $ 0.950 < \Omega_\star < 0.975 $
 - $ 0.25 < \Delta_p^2 < 0.45 $
 - $ -2.35 < n_p < -2.25 $
 - $ -0.265 < \alpha_p < -0.165 $

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import camb_cosmo
import fit_linP
import sim_params_cosmo
import sim_params_space
import read_genic
import write_config
import latin_hypercube
import corner

### Define parameter space, and choose one simulation

In [2]:
# get list of simulation parameters
add_slope=False
add_running=False
add_mu_H=False
param_space=sim_params_space.SimulationParameterSpace(add_slope=add_slope,add_running=add_running,add_mu_H=add_mu_H)
params=param_space.params
for key,param in params.items():
    print(key,param)

Om_star {'ip': 0, 'min_val': 0.95, 'max_val': 0.975, 'z_star': 3.0, 'latex': '$\\Omega_\\star$'}
Delta2_star {'ip': 1, 'min_val': 0.25, 'max_val': 0.45, 'z_star': 3.0, 'kp_Mpc': 0.7, 'latex': '$\\Delta^2_\\star$'}
mu_He {'ip': 2, 'min_val': 0.5, 'max_val': 2.0, 'latex': '$\\mu_{\\rm He}$'}


In [3]:
# get pivot point
z_star=params['Om_star']['z_star']
kp_Mpc=params['Delta2_star']['kp_Mpc']
print('z_star =',z_star)
print('kp_Mpc =',kp_Mpc)
Npar=len(params)
param_limits=np.empty([Npar,2])
for key,param in params.items():
    ip=param['ip']
    param_limits[ip][0]=param['min_val']
    param_limits[ip][1]=param['max_val']

z_star = 3.0
kp_Mpc = 0.7


In [4]:
# generate Latin hypercube 
nsamples=1
seed=101
cube=latin_hypercube.get_hypercube_samples(param_limits, nsamples, prior_points = None, seed=seed)
sim_params=cube[0]
print(sim_params)

[ 0.9625  0.35    1.25  ]


###  Setup fiducial cosmology, use it to find simulation cosmology, write all relevant files

In [5]:
# get fiducial cosmology
cosmo_fid = camb_cosmo.get_cosmology()
camb_cosmo.print_info(cosmo_fid)
# setup fiducial linear power model
linP_model_fid=fit_linP.LinearPowerModel(cosmo_fid,z_star=z_star,k_units='Mpc',kp=kp_Mpc)
print('fiducial linear power parameters',linP_model_fid.get_params())

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_c h^2 = 1.2000E-01, Omega_k = 0.0000E+00, Omega_nu h^2 = 0.0000E+00, T_CMB = 2.7255E+00, A_s = 2.1000E-09, n_s = 9.6500E-01, alpha_s = 0.0000E+00
fiducial linear power parameters {'f_star': 0.98136945578065649, 'g_star': 0.9677508579459803, 'Delta2_star': 0.36391885574433314, 'n_star': -2.3025651886411502, 'alpha_star': -0.21497062246191764}


In [6]:
# make sure simulation folder exists
sim_dir='test_single_pair'+'/'
write_config.mkdir_if_not_exists(sim_dir)
# setup cosmology model
cosmo_sim=sim_params_cosmo.cosmo_from_sim_params(params,sim_params,linP_model_fid,verbose=False)
# make a different folder for each simulation in the pair
plus_dir=sim_dir+'/sim_plus/'
write_config.mkdir_if_not_exists(plus_dir)
minus_dir=sim_dir+'/sim_minus/'
write_config.mkdir_if_not_exists(minus_dir)
# write GenIC and MP-Gadget parameters, for both simulations in pair
write_config.write_genic_file(plus_dir,cosmo_sim,paired=False)
zs=write_config.write_gadget_file(plus_dir,cosmo_sim)
write_config.write_genic_file(minus_dir,cosmo_sim,paired=True)
_=write_config.write_gadget_file(minus_dir,cosmo_sim)    
# construct linear power model and store in JSON format
linP_model_sim=fit_linP.LinearPowerModel(cosmo_sim,z_star=z_star,k_units='Mpc',kp=kp_Mpc)
linP_zs=write_config.write_sim_json_file(sim_dir,params,sim_params,linP_model_sim,zs)
# store information for plots
info_cosmo={'cosmo':cosmo_sim,'sim_params':sim_params,'linP_params':linP_model_sim.get_params()}

In [7]:
print(info_cosmo['sim_params'])
print(info_cosmo['linP_params'])

[ 0.9625  0.35    1.25  ]
{'f_star': 0.97868839087671933, 'g_star': 0.9629207127737258, 'Delta2_star': 0.34999999999999876, 'n_star': -2.3025673560744395, 'alpha_star': -0.21497519531101889}
